# Task_3: Social_Distancing_detector

Social distancing is a method used to control the spread of contagious diseases. It implies that people physically distance themselves from one another, reducing close contact, and thereby reducing the spread of a contagious disease (such as the COVID-19 Disease).

In this tutorial, I will implement a COVID-19 social distancing detector using yolov3.


In [9]:
# import libraries
import config
from detection import detect_people
from scipy.spatial import distance as dist 
import numpy as np
import argparse
from numba import cuda
import imutils
import cv2
import os

In [10]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--video-path", type=str, default="video.mp4", help="path to (optional) input video file")
ap.add_argument("-o", "--video-output-path", type=str, default="output.avi", help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1, help="whether or not output frame should be displayed")
args, unknown = ap.parse_known_args()

In [11]:
# load class labels
LABELS = "./coco.names"

In [12]:
cg = "./yolov3.cfg"
weights = "./yolov3.weights"

In [13]:
# load YOLO object detector
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(cg, weights)

[INFO] loading YOLO from disk...


In [14]:
# set CUDA s the preferable backend and target
print("[INFO] setting preferable backend and target to CUDA...")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

[INFO] setting preferable backend and target to CUDA...


In [15]:
# determine only the "output" layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

In [16]:
if args.video_path:
# initialize the video stream and pointer to output video file
    print("[INFO] accessing video stream...")
# open input video if available else webcam stream
    vs = cv2.VideoCapture(args.video_path if args.video_path else 0)
    writer = None

    # loop over the frames from the video stream
    while True:
        # read the next frame from the input video
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then that's the end fo the stream 
        if not grabbed:
            break

        # resize the frame and then detect people (only people) in it
        frame = imutils.resize(frame, width=700)
        results = detect_people(frame, net, ln)

        # initialize the set of indexes that violate the minimum social distance
        violate = set()

        # ensure there are at least two people detections (required in order to compute the
        # the pairwise distance maps)
        if len(results) >= 2:
            # extract all centroids from the results and compute the Euclidean distances
            # between all pairs of the centroids
            centroids = np.array([r[2] for r in results])
            D = dist.cdist(centroids, centroids, metric="euclidean")

            # loop over the upper triangular of the distance matrix
            for i in range(0, D.shape[0]):
                for j in range(i+1, D.shape[1]):
                    # check to see if the distance between any two centroid pairs is less
                    # than the configured number of pixels
                    if D[i, j] < config.MIN_DISTANCE:
                        # update the violation set with the indexes of the centroid pairs
                        violate.add(i)
                        violate.add(j)

        # loop over the results
        for (i, (prob, bbox, centroid)) in enumerate(results):
            # extract teh bounding box and centroid coordinates, then initialize the color of the annotation
            (startX, startY, endX, endY) = bbox
            (cX, cY) = centroid
            color = (0, 255, 0)

            # if the index pair exists within the violation set, then update the color
            if i in violate:
                color = (0, 0, 255)

            # draw (1) a bounding box around the person and (2) the centroid coordinates of the person
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.circle(frame, (cX, cY), 5, color, 1)

        # draw the total number of social distancing violations on the output frame
        text = "Social Distancing Violations: {}".format(len(violate))
        cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)

        # check to see if the output frame should be displayed to the screen
        if args.display > 0:
            # show the output frame
            cv2.imshow("Output", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the 'q' key is pressed, break from the loop
            if key == ord("q"):
                break

        # if  the video writer has not been  as none
        if writer is None:
            # initialize the video writer
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(args.video_output_path, fourcc, 25, (frame.shape[1], frame.shape[0]), True)

[INFO] accessing video stream...
